# Makam pitch sequence classification with LSTM

### Library importing for file reading and preprocessing

### Preprocessing pitch files

Before proceeding, the pitch files on the CompMusic Dunya makam corpus need to be converted in the quantized pitch series encoding constructed as described in the pseudocode below.

In [1]:
import glob
import os
import numpy as np
from keras.preprocessing import sequence
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


### File reading, octave folded encoding

In [68]:
#quantized octave-folded pitch file directory
ofq_read_dir = "./octfold_qdata/" 

#Makam list for more efficient file searching during label retrieval
makams = ["Acemasiran", "Acemkurdi", "Bestenigar", "Beyati", "Hicaz", "Hicazkar", "Huseyni", "Huzzam", "Karcigar", "Kurdilihicazkar", "Mahur", "Muhayyer", "Neva", "Nihavent", "Rast", "Saba", "Segah", "Sultaniyegah", "Suzinak", "Ussak"]

all_qpitch = [] #array holding all quantized pitch series as strings
y = [] #holds makam labels
max_length = 0
for root, dirs, files in os.walk(ofq_read_dir):
    for name in files:
        if '.pitch' in name:
            #retrieve label from parent of original path
            for makam in makams:
                if (os.path.isfile("./otmm_makam_recognition_dataset/data/" + makam + "/" + name) == True):
                    y.append(makams.index(makam))
                    break
            
            with open(os.path.join(root, name)) as f:
                content = f.read()
                all_qpitch.append(content)
print("File number:", len(y))
print("Array length:", len(all_qpitch))

File number: 1000
[4, 8, 19, 13, 14, 0, 12, 17, 18, 13, 13, 4, 0, 3, 17, 2, 14, 4, 13, 10, 6, 19, 15, 8, 14, 9, 11, 1, 5, 5, 4, 2, 14, 18, 4, 10, 16, 11, 15, 5, 17, 10, 6, 9, 12, 3, 1, 14, 14, 14, 6, 5, 12, 17, 15, 17, 17, 14, 1, 14, 14, 16, 11, 7, 11, 5, 4, 2, 18, 6, 15, 7, 7, 8, 14, 1, 2, 13, 9, 3, 9, 16, 1, 8, 12, 13, 11, 13, 9, 9, 18, 14, 14, 1, 16, 18, 6, 1, 0, 6, 0, 9, 18, 14, 3, 14, 12, 3, 3, 4, 15, 9, 3, 18, 13, 14, 7, 5, 14, 19, 18, 10, 14, 15, 18, 3, 15, 3, 16, 8, 19, 15, 5, 5, 9, 1, 16, 7, 2, 5, 0, 5, 11, 10, 17, 10, 4, 0, 11, 15, 19, 17, 9, 14, 13, 0, 12, 15, 6, 0, 7, 16, 16, 16, 5, 1, 3, 7, 4, 12, 11, 10, 13, 11, 8, 12, 16, 17, 13, 15, 5, 14, 6, 2, 14, 14, 14, 11, 7, 19, 5, 15, 8, 11, 10, 10, 14, 17, 17, 0, 4, 2, 15, 19, 6, 17, 2, 7, 19, 8, 2, 9, 14, 6, 5, 15, 2, 6, 11, 15, 6, 5, 17, 16, 6, 6, 7, 19, 19, 10, 9, 4, 9, 14, 2, 10, 5, 19, 16, 14, 2, 9, 9, 11, 7, 1, 8, 9, 0, 12, 8, 8, 9, 0, 13, 11, 13, 5, 8, 1, 0, 8, 2, 17, 7, 0, 15, 11, 13, 2, 4, 13, 16, 11, 9, 14, 11, 17, 8, 

### Preprocessing
Pading input sequences

In [48]:
#Tokenization
top_k = 636 #53*4*3
tokenizer = Tokenizer(num_words=top_k, split='\n')
tokenizer.fit_on_texts(all_qpitch)
seqs = tokenizer.texts_to_sequences(all_qpitch)
print(seqs)

In [49]:
#Padding
X = sequence.pad_sequences(seqs, padding='post')
print(X)

[[15  5  2 ...  0  0  0]
 [11  7  2 ...  0  0  0]
 [47  4  1 ...  0  0  0]
 ...
 [12  8  1 ...  0  0  0]
 [14  7  1 ...  0  0  0]
 [33  4  1 ...  0  0  0]]


### Library importing for deep learning

In [50]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split

### Train - Test split

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Building the LSTM

In [65]:
#Word embedding
embedding_vector_len = 7 #max pitch value length: 4, comma, significance value of length 1, newline 

#Network topology
model = Sequential()

model.add(Embedding(top_k, embedding_vector_len, input_length=len(X[0])))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [66]:
#Compile model
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 213393, 7)         4452      
_________________________________________________________________
dropout_9 (Dropout)          (None, 213393, 7)         0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               43200     
_________________________________________________________________
dropout_10 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 47,753
Trainable params: 47,753
Non-trainable params: 0
_________________________________________________________________


In [70]:
#Fitting
model.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10


In [ ]:
#Model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print(scores)